<a href="https://colab.research.google.com/github/women-in-ai-ireland/May-2022-WaiXCollins-Airplane-Crash-Investigation/blob/main/dataset/geopyLoc_Route.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import numpy as np
df=pd.read_csv("https://github.com/women-in-ai-ireland/May-2022-WaiXCollins-Airplane-Crash-Investigation/raw/main/dataset/Airplane_Crashes_and_Fatalities_Since_1908_20190820105639.csv")

In [34]:
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent="geoapiExercises")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def loc_to_geo(row):
  if not (isinstance(row["geopyLoc"], geopy.location.Location)):
    loc=row["Location"]
    # remove redundant keywords and clear the text
    try:
      loc=loc.lower().replace("near","").replace("off","").replace("over the","").strip()
    except:
      return None

    return geocode(loc,language='en',addressdetails=True)


def route_to_geo(row):
  if not (isinstance(row["geopyRoute"], list)):
    try:
      # we assume that the origin and destination are devided by hyphen
      route_list=row["Route"].split('-')
    except:
      return None

    # if the output have one element, it means we only have one point (no hyphen)
    # which is not enough to draw the route
    if len(route_list)==1:
      return None

    geo_route_list=[]
    for loc in route_list:
      geoloc=geocode(loc,language='en',addressdetails=True)
      # if any of route points does not return a geopy location, we skip the entire route
      if not geoloc:
        return None
      else:
        geo_route_list.append(geoloc)

    return geo_route_list

In [ ]:
# create an empty column if it is not existed
if not("geopyLoc" in (df.columns)):
  df["geopyLoc"]=None

# apply desired function (loc_to_geo) to rows which do not have value in "geopyLoc"
# this allows us to break the task and update only remaining records
# and insert the output to a pandas.series called "result"
result=df[df["geopyLoc"].isna()].apply(loc_to_geo,axis="columns")
df.loc[result.index,"geopyLoc"]=result

In [ ]:
# create an empty column if it is not existed
if not("geopyRoute" in (df.columns)):
  df["geopyRoute"]=None

result=df[df["geopyRoute"].isna()].apply(route_to_geo,axis="columns")
df.loc[result.index,"geopyRoute"]=result

In [43]:
# save as pickle(pkl) to preserve the data types
df.to_pickle("Airplane_Crashes_and_Fatalities_Since_1908_20190820105639.pkl")